In [1]:
import pandas as pd
import numpy as np

In [2]:
num_cols = ['price','con','snow', 'ice', 'dur']

df1 = pd.read_csv('data.csv', header=3)[:17]
df1[num_cols] = df1[num_cols].apply(pd.to_numeric)
df1.head()

,No,System,mount,price,con,snow,ice,dur,Accegrade
0,1,SK,F,206,1.9,1.4,1.8,2.7,F
1,2,SRK,F or R,520,2.1,0.8,3.8,2.3,F
2,3,SK,F,160,1.7,1.9,1.6,3.7,F
3,4,SK,F,213,1.7,2.0,2.4,3.4,F
4,5,SMS,F or R,598,1.6,2.4,7.0,2.8,F


In [3]:
df2 = pd.read_csv('data.csv', header=3)[22:].reset_index(drop=True)
df2[num_cols] = df2[num_cols].apply(pd.to_numeric)

In [4]:
def split_last_3(data):
    x, y = data.drop(columns=['Accegrade', 'No']), data['Accegrade']
    x_train, x_test, y_train, y_test = x[:-3], x[-3:], y[:-3], y[-3:] 
    return x_train, x_test, y_train, y_test

In [5]:
def calc_sim(test, train):
    sim = []
    for i, j in enumerate(test):
        if i < 2:
            if j == train[i]:
                sim.append(j)
            else:
                sim.append('*')
        else:
            sim.append((min(j, train[i]), max(j, train[i])))
    return sim

def check_sim(train, sim):
    categorical = []
    for k, i in enumerate(sim[:2]):
        if train[k] == i or i =='*':
            categorical.append(True)
        else:
            categorical.append(False)
            
    numerical = []
    for k, i in enumerate(sim[2:]):
        if train[k + 2] <= i[1] and train[k + 2] >= i[0]:
            numerical.append(True)
        else:
            numerical.append(False)
    return np.all(categorical + numerical)
    
def calc_extsim(x_train, sim):
    idxs = []
    for i in range(len(x_train)):
        idxs.append(check_sim(x_train.iloc[i], sim))
    return idxs

def predict(x_train, y_train, x_test, y_test):
    preds = []
    for i in range(len(x_test)):
        for j in range(len(x_train)):
            simm = calc_sim(x_test.iloc[i], x_train.iloc[j])
            targets = y_train.iloc[calc_extsim(x_train, simm)]
            if np.all(targets == 'F') and len(targets.index.values) != 0:
                print(f'IntSec for test object {i}:', simm)
                print('Extension:', targets.index.values)
                preds.append('F')
                break
            elif np.all(targets == 'T') and len(targets.index.values) != 0:
                print(f'IntSec for test object {i}:', simm)
                print('Extension:', targets.index.values)
                preds.append('T')
                break
            else:
                print('Trying', simm)
                if j == len(x_train)-1:
                    preds.append('Undetermined')
                    break
    return preds, np.round(np.mean(preds == y_test), 2)

## №1

In [6]:
x_train, x_test, y_train, y_test = split_last_3(df1)

In [7]:
preds, acc = predict(x_train, y_train, x_test, y_test)

print('\nAccrucay:', acc,'\nTrue values:', y_test.values, '\nPredictions:', preds)

IntSec for test object 0: ['SK', 'F', (99, 206), (1.9, 2.8), (1.4, 2.2), (1.8, 2.5), (2.7, 4.0)]
Extension: [0 5]
IntSec for test object 1: ['SK', 'F', (140, 206), (1.9, 2.6), (1.4, 2.3), (1.8, 3.3), (2.7, 3.4)]
Extension: [0]
IntSec for test object 2: ['SK', 'F', (206, 215), (1.9, 2.3), (1.4, 3.8), (1.8, 4.8), (2.3, 2.7)]
Extension: [0]

Accrucay: 0.0 
True values: ['T' 'T' 'T'] 
Predictions: ['F', 'F', 'F']


## №2

In [8]:
x_train, x_test, y_train, y_test = split_last_3(df2)

In [9]:
preds, acc = predict(x_train, y_train, x_test, y_test)

print('\nAccrucay:', acc,'\nTrue values:', y_test.values, '\nPredictions:', preds)

IntSec for test object 0: ['SK', 'F', (99, 149), (1.9, 2.8), (2.2, 2.5), (2.5, 4.0), (3.8, 4.0)]
Extension: [0]
IntSec for test object 1: ['SK', 'F', (140, 149), (1.9, 2.6), (2.3, 2.5), (3.3, 4.0), (3.4, 3.8)]
Extension: [0]
IntSec for test object 2: ['SK', 'F', (149, 215), (1.9, 2.3), (2.5, 3.8), (4.0, 4.8), (2.3, 3.8)]
Extension: [0]

Accrucay: 1.0 
True values: ['T' 'T' 'T'] 
Predictions: ['T', 'T', 'T']


## №3

In [10]:
X, y = df2.drop(columns=['Accegrade', 'No']), df2['Accegrade']

In [11]:
accs = []
test_idx = np.arange(0, 3)
for i in range(0, 5):
    print(f'Fold {i+1}')
    test_idx += 3 * (i != 0)
    train_idx = ~X.index.isin(test_idx)
    x_train, x_test, y_train, y_test = X.iloc[train_idx], X.iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
    p, a = predict(x_train, y_train, x_test, y_test)
    print('\nPedicted vs True')
    print(p)
    print(y_test.values)
    print('Accuracy:', a)
    print('\n------------\n')
    accs.append(a)

print('Mean accuracy:', np.mean(accs))

Fold 1
IntSec for test object 0: ['SK', 'F', (149, 213), (1.7, 1.9), (2.0, 2.5), (2.4, 4.0), (3.4, 3.8)]
Extension: [3]
IntSec for test object 1: ['*', '*', (213, 520), (1.7, 2.1), (0.8, 2.0), (2.4, 3.8), (2.3, 3.4)]
Extension: [3]
Trying ['*', '*', (213, 389), (1.7, 2.0), (2.0, 2.2), (2.4, 3.3), (3.4, 4.3)]
Trying ['*', '*', (389, 598), (1.6, 2.0), (2.2, 2.4), (3.3, 7.0), (2.8, 4.3)]
Trying ['*', '*', (109, 389), (2.0, 2.0), (1.9, 2.2), (2.4, 3.3), (3.7, 4.3)]
Trying ['SRK', '*', (325, 389), (2.0, 2.0), (2.1, 2.2), (3.2, 3.3), (2.8, 4.3)]
IntSec for test object 2: ['*', '*', (389, 498), (1.5, 2.0), (2.2, 3.3), (3.3, 3.5), (2.0, 4.3)]
Extension: [ 7 10]

Pedicted vs True
['F', 'F', 'T']
['T' 'F' 'T']
Accuracy: 0.67

------------

Fold 2
IntSec for test object 0: ['SK', 'F', (149, 213), (1.7, 1.9), (2.0, 2.5), (2.4, 4.0), (3.4, 3.8)]
Extension: [0]
IntSec for test object 1: ['*', '*', (149, 598), (1.6, 1.9), (2.4, 2.5), (4.0, 7.0), (2.8, 3.8)]
Extension: [0]
IntSec for test object 2: ['

## №4

In [12]:
def calc_sim(test, train):
    sim = []
    for i, j in enumerate(test):
        if i < 2:
            if j == train[i]:
                sim.append(j)
            else:
                sim.append('*')
        else:
            sim.append((min(j, train[i]), np.inf))
    return sim

In [13]:
test_idx = [14]
train_idx = ~X.index.isin(test_idx)

In [14]:
x_train, x_test, y_train, y_test = X.iloc[train_idx], X.iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]

In [15]:
preds, acc = predict(x_train, y_train, x_test, y_test)

print('\nAccrucay:', acc,'\nTrue values:', y_test.values, '\nPredictions:', preds)

IntSec for test object 0: ['SK', 'F', (99, inf), (1.9, inf), (2.2, inf), (2.5, inf), (3.8, inf)]
Extension: [0]

Accrucay: 1.0 
True values: ['T'] 
Predictions: ['T']


## №5

In [16]:
def calc_sim(test, train):
    sim = []
    for i, j in enumerate(test):
        if i < 2:
            if j == train[i]:
                sim.append(j)
            else:
                sim.append('*')
        else:
            sim.append((max(j, train[i]), np.inf))
    return sim

In [17]:
preds, acc = predict(x_train, y_train, x_test, y_test)

print('\nAccrucay:', acc,'\nTrue values:', y_test.values, '\nPredictions:', preds)

Trying ['SK', 'F', (149, inf), (2.8, inf), (2.5, inf), (4.0, inf), (4.0, inf)]
Trying ['*', '*', (520, inf), (2.8, inf), (2.2, inf), (3.8, inf), (4.0, inf)]
Trying ['*', '*', (389, inf), (2.8, inf), (2.2, inf), (3.3, inf), (4.3, inf)]
Trying ['SK', 'F', (213, inf), (2.8, inf), (2.2, inf), (2.5, inf), (4.0, inf)]
Trying ['*', '*', (598, inf), (2.8, inf), (2.4, inf), (7.0, inf), (4.0, inf)]
Trying ['SK', 'F', (109, inf), (2.8, inf), (2.2, inf), (2.5, inf), (4.0, inf)]
Trying ['*', '*', (325, inf), (2.8, inf), (2.2, inf), (3.2, inf), (4.0, inf)]
Trying ['*', '*', (498, inf), (2.8, inf), (3.3, inf), (3.5, inf), (4.0, inf)]
Trying ['*', '*', (396, inf), (2.8, inf), (2.2, inf), (3.1, inf), (4.0, inf)]
Trying ['SK', 'F', (160, inf), (2.8, inf), (2.2, inf), (2.5, inf), (4.0, inf)]
Trying ['*', '*', (389, inf), (2.8, inf), (2.2, inf), (3.3, inf), (4.3, inf)]
Trying ['*', 'F', (298, inf), (2.8, inf), (2.3, inf), (3.3, inf), (4.0, inf)]
Trying ['SK', 'F', (206, inf), (2.8, inf), (2.2, inf), (2.5,